In [2]:
import utils
import google.generativeai as genai

# --------------------------
# Basic moderation (Gemini)
# --------------------------
def moderate_input(user_input, model="gemini-1.5-flash"):
    """
    Simple safety check using Gemini. 
    (Gemini already filters unsafe inputs internally, so this is just a mock moderation step).
    """
    system_message = """
    Check if the input is safe for a customer service chatbot.
    If unsafe, reply with "FLAGGED". Otherwise, reply "OK".
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input}
    ]
    response = utils.get_completion_from_messages(messages, model)
    return "FLAGGED" in response.upper()

# --------------------------
# Evaluation step
# --------------------------
def evaluate_response(user_input, final_response, model="gemini-1.5-flash"):
    """
    Ask Gemini to judge if the response sufficiently answers the query.
    Returns 'Y' or 'N'.
    """
    delimiter = "```"
    system_message = """
    You are an evaluator for a grocery store chatbot. 
    Respond only with Y (sufficient) or N (insufficient).
    """
    eval_prompt = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the customer message?
    Reply with Y or N.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": eval_prompt}
    ]
    return utils.get_completion_from_messages(messages, model).strip()

# --------------------------
# Main pipeline
# --------------------------
def process_user_message(user_input, all_messages, model="gemini-1.5-flash", debug=True):
    delimiter = "```"

    # Step 1: Moderation
    if moderate_input(user_input, model):
        if debug: print("Step 1: Input flagged ")
        return "Sorry, we cannot process this request.", all_messages
    if debug: print("Step 1: Input passed moderation ")

    # Step 2: Extract categories/products
    products = utils.get_products()
    products_and_category = {}
    for p in products.values():
        products_and_category.setdefault(p["category"], []).append(p["name"])
    raw_extraction = utils.find_category_and_product(user_input, products_and_category)
    category_and_product_list = utils.read_string_to_list(raw_extraction)
    if debug: print("Step 2: Extracted entities ", category_and_product_list)

    # Step 3: Lookup product info
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product info ")

    # Step 4: Generate chatbot reply
    system_message = """
    You are a customer service assistant for an online grocery store. 
    Respond in a friendly and helpful tone, with concise answers. 
    Make sure to ask relevant follow-up questions.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Relevant product information:\n{product_information}"}
    ]
    final_response = utils.get_completion_from_messages(all_messages + messages, model)
    all_messages = all_messages + messages[1:]
    if debug: print("Step 4: Generated chatbot reply ")

    # Step 5: Evaluate the response
    evaluation = evaluate_response(user_input, final_response, model)
    if debug: print("Step 5: Evaluation result:", evaluation)

    if "Y" in evaluation.upper():
        if debug: print("Step 6: Response approved ")
        return final_response, all_messages
    else:
        if debug: print("Step 6: Response disapproved ")
        return "I'm not confident in my response. Let me connect you to a human assistant.", all_messages


In [4]:
#from pipeline import process_user_message

# Conversation context
context = [{"role": "system", "content": "You are Grocery Store Assistant"}]

query = "Tell me about bananas and milk"
response, context = process_user_message(query, context, debug=True)

print("\n Final Chatbot Reply:", response)


Step 1: Input passed moderation 
Error parsing JSON: ```json
[
  {
    "category": "Fruits",
    "products": ["Bananas"]
  },
  {
    "category": "Dairy",
    "products": ["Milk"]
  }
]
```
Step 2: Extracted entities  []
Step 3: Looked up product info 
Step 4: Generated chatbot reply 
Step 5: Evaluation result: N
Step 6: Response disapproved 

 Final Chatbot Reply: I'm not confident in my response. Let me connect you to a human assistant.
